<a href="https://colab.research.google.com/github/johnnykoo84/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

## 1. Load and clean the data

In [0]:
### YOUR CODE STARTS HERE

In [0]:
# imports
import pandas as pd
from scipy.stats import ttest_ind

In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-01-30 12:02:53--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2020-01-30 12:02:54 (130 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

In [0]:
df = pd.read_csv('house-votes-84.data',
                 header=None,
                 names=column_headers,
                 na_values='?')

In [6]:
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [7]:
df.shape

(435, 17)

In [8]:
# recode votes as numeric
df = df.replace({'y': 1, 'n': 0})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [9]:
# how did Republican vote?
rep = df[df['party'] == 'republican']
rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [10]:
# how did Democrats vote?
dem = df[df['party'] == 'democrat']
dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [0]:
import numpy as np

In [12]:
# the percentage of republicans who voted "yes" (1) 
# on the handicapped-infants bill
rep['handicapped-infants'].sum()/len(rep)


# len() is counting NaN values too!

0.18452380952380953

In [13]:
# Remove NaN values from this column
col = rep['handicapped-infants']
np.isnan(col)
hi_no_nans = col[~np.isnan(col)]
# The same column as before, but I've dropped the NaN values
hi_no_nans

0      0.0
1      0.0
7      0.0
8      0.0
10     0.0
      ... 
427    0.0
430    0.0
432    0.0
433    0.0
434    0.0
Name: handicapped-infants, Length: 165, dtype: float64

In [14]:
hi_no_nans.sum()/len(hi_no_nans)

0.18787878787878787

In [15]:
# Average rate of voting 'yes' on the handicapped-infants
rep['handicapped-infants'].mean()

0.18787878787878787

In [16]:
# let's do the same for Democrats
# Remove NaN values from this column
col = dem['handicapped-infants']
np.isnan(col)
hi_no_nans_dem = col[~np.isnan(col)]
# The same column as before, but I've dropped the NaN values
hi_no_nans_dem

3      0.0
4      1.0
5      0.0
6      0.0
9      1.0
      ... 
424    0.0
425    0.0
426    1.0
429    1.0
431    0.0
Name: handicapped-infants, Length: 258, dtype: float64

In [17]:
hi_no_nans_dem.sum()/len(hi_no_nans_dem)

0.6046511627906976

## p values
let's find statistics and p values for all column sample data

In [27]:
df.columns

Index(['party', 'handicapped-infants', 'water-project', 'budget',
       'physician-fee-freeze', 'el-salvador-aid', 'religious-groups',
       'anti-satellite-ban', 'aid-to-contras', 'mx-missile', 'immigration',
       'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free',
       'south-africa'],
      dtype='object')

In [38]:
# let's see all statistics and p-value for all issues

result_handicapped_infants = ttest_ind(rep['handicapped-infants'], dem['handicapped-infants'], nan_policy='omit')
mean_rep_handicapped = rep['handicapped-infants'].mean()
mean_dem_handicapped = dem['handicapped-infants'].mean()

result_water_project = ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit')
mean_rep_water_project = rep['water-project'].mean()
mean_dem_water_project = dem['water-project'].mean()

result_budget = ttest_ind(rep['budget'], dem['budget'], nan_policy='omit')
mean_rep_budget = rep['budget'].mean()
mean_dem_budget = dem['budget'].mean()

result_physician_fee_freeze = ttest_ind(rep['physician-fee-freeze'], dem['physician-fee-freeze'], nan_policy='omit')
mean_rep_physician_fee_freeze = rep['physician-fee-freeze'].mean()
mean_dem_physician_fee_freeze = dem['physician-fee-freeze'].mean()

result_el_salvador_aid = ttest_ind(rep['el-salvador-aid'], dem['el-salvador-aid'], nan_policy='omit')
mean_rep_el_salvador_aid = rep['el-salvador-aid'].mean()
mean_dem_el_salvador_aid = dem['el-salvador-aid'].mean()

result_religious_groups = ttest_ind(rep['religious-groups'], dem['religious-groups'], nan_policy='omit')
mean_rep_religious_groups = rep['religious-groups'].mean()
mean_dem_religious_groups = dem['religious-groups'].mean()

result_anti_satellite_ban = ttest_ind(rep['anti-satellite-ban'], dem['anti-satellite-ban'], nan_policy='omit')
mean_rep_anti_satellite_ban = rep['anti-satellite-ban'].mean()
mean_dem_anti_satellite_ban = dem['anti-satellite-ban'].mean()

result_aid_to_contras = ttest_ind(rep['aid-to-contras'], dem['aid-to-contras'], nan_policy='omit')
mean_rep_aid_to_contras = rep['aid-to-contras'].mean()
mean_dem_aid_to_contras = dem['aid-to-contras'].mean()

result_mx_missile = ttest_ind(rep['mx-missile'], dem['mx-missile'], nan_policy='omit')
mean_rep_mx_missile = rep['mx-missile'].mean()
mean_dem_mx_missile = dem['mx-missile'].mean()

result_immigration = ttest_ind(rep['immigration'], dem['immigration'], nan_policy='omit')
mean_rep_immigration = rep['immigration'].mean()
mean_dem_immigration = dem['immigration'].mean()

result_synfuels = ttest_ind(rep['synfuels'], dem['synfuels'], nan_policy='omit')
mean_rep_synfuels = rep['synfuels'].mean()
mean_dem_synfuels = dem['synfuels'].mean()

result_education = ttest_ind(rep['education'], dem['education'], nan_policy='omit')
mean_rep_education = rep['education'].mean()
mean_dem_education = dem['education'].mean()

result_right_to_sue = ttest_ind(rep['right-to-sue'], dem['right-to-sue'], nan_policy='omit')
mean_rep_right_to_sue = rep['right-to-sue'].mean()
mean_dem_right_to_sue = dem['right-to-sue'].mean()

result_crime = ttest_ind(rep['crime'], dem['crime'], nan_policy='omit')
mean_rep_crime = rep['crime'].mean()
mean_dem_crime = dem['crime'].mean()

result_duty_free = ttest_ind(rep['duty-free'], dem['duty-free'], nan_policy='omit')
mean_rep_duty_free = rep['duty-free'].mean()
mean_dem_duty_free = dem['duty-free'].mean()

result_south_africa = ttest_ind(rep['south-africa'], dem['south-africa'], nan_policy='omit')
mean_rep_south_africa = rep['south-africa'].mean()
mean_dem_south_africa = dem['south-africa'].mean()

print('rep mean: ', mean_rep_handicapped)
print('dem mean: ', mean_dem_handicapped)
print('handicapped_infants\n', result_handicapped_infants)

print('rep mean: ', mean_rep_water_project)
print('dem mean: ', mean_dem_water_project)
print('water_project\n', result_water_project)

print('rep mean: ', mean_rep_budget)
print('dem mean: ', mean_dem_budget)
print('budget\n', result_budget)

print('rep mean: ', mean_rep_physician_fee_freeze)
print('dem mean: ', mean_dem_physician_fee_freeze)
print('physician_fee_freeze\n', result_physician_fee_freeze)

print('rep mean: ', mean_rep_el_salvador_aid)
print('dem mean: ', mean_dem_el_salvador_aid)
print('el_salvador_aid\n', result_el_salvador_aid)

print('rep mean: ', mean_rep_religious_groups)
print('dem mean: ', mean_dem_religious_groups)
print('religious_groups\n', result_religious_groups)

print('rep mean: ', mean_rep_anti_satellite_ban)
print('dem mean: ', mean_dem_anti_satellite_ban)
print('anti_satellite_ban\n', result_anti_satellite_ban)

print('rep mean: ', mean_rep_aid_to_contras)
print('dem mean: ', mean_dem_aid_to_contras)
print('aid_to_contras\n', result_aid_to_contras)

print('rep mean: ', mean_rep_mx_missile)
print('dem mean: ', mean_dem_mx_missile)
print('mx_missile\n', result_mx_missile)

print('rep mean: ', mean_rep_immigration)
print('dem mean: ', mean_dem_immigration)
print('immigration\n', result_immigration)

print('rep mean: ', mean_rep_synfuels)
print('dem mean: ', mean_dem_synfuels)
print('synfuels\n', result_synfuels)

print('rep mean: ', mean_rep_education)
print('dem mean: ', mean_dem_education)
print('education\n', result_education)

print('rep mean: ', mean_rep_right_to_sue)
print('dem mean: ', mean_dem_right_to_sue)
print('right_to_sue\n', result_right_to_sue)

print('rep mean: ', mean_rep_crime)
print('dem mean: ', mean_dem_crime)
print('crime', result_crime)

print('rep mean: ', mean_rep_duty_free)
print('dem mean: ', mean_dem_duty_free)
print('duty_free\n', result_duty_free)

print('rep mean: ', mean_rep_south_africa)
print('dem mean: ', mean_dem_south_africa)
print('south_africa\n', result_south_africa)

rep mean:  0.18787878787878787
dem mean:  0.6046511627906976
handicapped_infants
 Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
rep mean:  0.5067567567567568
dem mean:  0.502092050209205
water_project
 Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
rep mean:  0.13414634146341464
dem mean:  0.8884615384615384
budget
 Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
rep mean:  0.9878787878787879
dem mean:  0.05405405405405406
physician_fee_freeze
 Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
rep mean:  0.9515151515151515
dem mean:  0.21568627450980393
el_salvador_aid
 Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
rep mean:  0.8975903614457831
dem mean:  0.47674418604651164
religious_groups
 Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)
rep mean:  0.24074074074074073
dem mean:  0.7722007722007722
anti_satellite_ban
 

rep mean:  0.18787878787878787
dem mean:  0.6046511627906976
handicapped_infants
 Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)

rep mean:  0.5067567567567568
dem mean:  0.502092050209205
water_project
 Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

rep mean:  0.13414634146341464
dem mean:  0.8884615384615384
budget
 Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

rep mean:  0.9878787878787879
dem mean:  0.05405405405405406
physician_fee_freeze
 Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)


rep mean:  0.9515151515151515
dem mean:  0.21568627450980393
el_salvador_aid
 Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)

rep mean:  0.8975903614457831
dem mean:  0.47674418604651164
religious_groups
 Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)

rep mean:  0.24074074074074073
dem mean:  0.7722007722007722
anti_satellite_ban
 Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)

rep mean:  0.15286624203821655
dem mean:  0.8288973384030418
aid_to_contras
 Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)

rep mean:  0.11515151515151516
dem mean:  0.7580645161290323
mx_missile
 Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47)

rep mean:  0.5575757575757576
dem mean:  0.4714828897338403
immigration
 Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)

rep mean:  0.1320754716981132
dem mean:  0.5058823529411764
synfuels
 Ttest_indResult(statistic=-8.293603989407588, pvalue=1.5759322301054064e-15)

rep mean:  0.8709677419354839
dem mean:  0.14457831325301204
education
 Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64)

rep mean:  0.8607594936708861
dem mean:  0.2896825396825397
right_to_sue
 Ttest_indResult(statistic=13.51064251060933, pvalue=1.2278581709672758e-34)

rep mean:  0.9813664596273292
dem mean:  0.35019455252918286
crime Ttest_indResult(statistic=16.342085656197696, pvalue=9.952342705606092e-47)

rep mean:  0.08974358974358974
dem mean:  0.6374501992031872
duty_free
 Ttest_indResult(statistic=-12.853146132542978, pvalue=5.997697174347365e-32)
 
rep mean:  0.6575342465753424
dem mean:  0.9351351351351351
south_africa
 Ttest_indResult(statistic=-6.849454815841208, pvalue=3.652674361672226e-11)

## 2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

1. handicapped-infants
2. budget
3. anti-satellite-ban
4. aid_to_contras
5. mx_missile
6. synfuels
7. duty_free
8. south_africa

## 3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

1. physician_fee_freeze
2. el_salvador_aid
3. religious_groups
4. education
5. right_to_sue
6. crime

## 4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

1. water_project